#### **1. Nutritional planning**
##### **a) *insert title***

##### **b) *insert title***

##### **c) *insert title***

##### **d) *insert title***

##### **e) *insert title***

#### **2. Transport problem**
##### **a) Linear optimization problem**

Let $x_{ij}$ be the quantity of units transported from source $i \ (S_i)$ to destination $j \ (D_j)$, where $i \in \{1, 2, 3\}$ and $j \in \{1, 2, 3, 4\}$.

**Objective function**

The goal is to minimize the total transportation cost $Z$:
$$\text{Minimize} \ Z = \sum_{i=1}^3 \sum_{j=1}^4 c_{ij}x_{ij}$$
Explicitly using the given costs $(c_{ij})$: $$\text{Min } Z = 10x_{11} + 0x_{12} + 20x_{13} + 11x_{14} + 12x_{21} + 7x_{22} + 9x_{23} + 20x_{24} + 0x_{31} + 14x_{32} + 16x_{33} + 18x_{34}$$

**Constraints**
1. The total volume transported from each source must not exceed the available supply ($A_i$): 
$$\sum_{j=1}^4 x_{ij} \le A_i \ \text{for} \ i = 1, 2, 3$$
- S1: $x_{11} + x_{12} + x_{13} + x_{14} \leq 20$
- S2: $x_{21} + x_{22} + x_{23} + x_{24} \leq 25$
- S3: $x_{31} + x_{32} + x_{33} + x_{34} \leq 15$

2. The total volume transported to each destination must equal the required demand ($V_j$):
$$\sum_{i=1}^3 x_{ij} = V_j \ \text{for} \ j = 1, 2, 3, 4$$
- D1: $x_{11} + x_{21} + x_{31} = 10$
- D2: $x_{12} + x_{22} + x_{32} = 15$
- D3: $x_{13} + x_{23} + x_{33} = 15$
- D4: $x_{14} + x_{24} + x_{34} = 20$

3. The transported quantities must be non-negative:
$$x_{ij} \ge 0 \ \text{for all} \ i, j$$


##### **b) Cheapest transportation plan**

First we define the parameters of the transportation problem and initialize the linear programming model using the `pulp` library. 

In [1]:
import pulp
from pulp import LpProblem, LpMinimize, LpVariable, LpStatus, lpSum, value

# Data Definition
Sources = ['S1', 'S2', 'S3']
Destinations = ['D1', 'D2', 'D3', 'D4']

# Cost matrix
costs = {
    'S1': {'D1': 10, 'D2': 0, 'D3': 20, 'D4': 11}, 
    'S2': {'D1': 12, 'D2': 7, 'D3': 9, 'D4': 20}, 
    'S3': {'D1': 0, 'D2': 14, 'D3': 16, 'D4': 18}
}

# Supply
supply = {'S1': 20,'S2': 25,'S3': 15}

# Demand
demand = {'D1': 10, 'D2': 15, 'D3': 15, 'D4': 20}

# Initializing the problem as a minimization problem
model = pulp.LpProblem("Transportation_Optimization", pulp.LpMinimize)

# Defining the decision variables 
x = pulp.LpVariable.dicts("x", 
                          ((i, j) for i in Sources for j in Destinations), 
                          lowBound=0,
                          cat= 'Continuous')



Next we formulate the objective function and add the supply and demand constraints to the model, followed by solving the problem. 

In [2]:
# Objective function: minimize total cost (Z = sum(c_ij * x_ij))
model += pulp.lpSum(costs [i][j] * x[(i, j)] for i in Sources for j in Destinations), "Total_Transportation_Cost"

# Supply constraints (total flow out of Source i <= Supply A_i)
for i in Sources: 
    model += pulp.lpSum(x[(i, j)] for j in Destinations) <= supply[i], f"Supply_Constraint_{i}"

# Demand constraints (Total flow into Destination j = Demand V_j)
for j in Destinations:
    model += pulp.lpSum(x[(i, j)] for i in Sources) == demand[j], f"Demand_Constraint_{j}"

# Solving the problem
model.solve()

# Checking the status of the solution
print(f"Solver Status: {pulp.LpStatus[model.status]}")

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /opt/anaconda3/lib/python3.13/site-packages/pulp/apis/../solverdir/cbc/osx/i64/cbc /var/folders/25/0hf3c7d931q0rktyfwqp3g_w0000gn/T/955fff463ff548fa8f039d8dbc8adc1c-pulp.mps -timeMode elapsed -branch -printingOptions all -solution /var/folders/25/0hf3c7d931q0rktyfwqp3g_w0000gn/T/955fff463ff548fa8f039d8dbc8adc1c-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 12 COLUMNS
At line 47 RHS
At line 55 BOUNDS
At line 56 ENDATA
Problem MODEL has 7 rows, 12 columns and 24 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Presolve 7 (0) rows, 12 (0) columns and 24 (0) elements
0  Obj 0 Primal inf 60 (4)
6  Obj 460
Optimal - objective value 460
Optimal objective 460 - 6 iterations time 0.002
Option for printingOptions changed from normal to all
Total time (CPU seconds):       0.00   (Wallclock seconds):       0.02

Solver Status: Opt

The problem was successfully solved. The optimal solution yields the minimum total cost and the optimal flow plan, which respects all supply and demand limitations. We can now print the final answer, which is the minimal cost and allocation of goods for each route. 

In [3]:
# Minimum total costs
if pulp.LpStatus[model.status] == "Optimal":
    min_cost = pulp.value(model.objective) 
    print("\n==================================================")
    print (f"The minimum required transportation cost (Z) is: €{min_cost:,.2f}")
    print("==================================================")

    # Optimal Transport Plan (Flows > 0)
    print("\nOptimal Transportation Plan (Flows > 0):")

    # A header for the table output
    print("{:<15} {:<15} {:<10}" .format("Source", "Destination", "Units Sent")) 
    print("-" * 40)

    for i in Sources: 
        for j in Destinations: 
            amount = pulp.value(x[(i,j)])
            if amount > 0:
                print("{:<15}" "{:<15}" "{:<10.0f}" .format(i, j, amount))
else:
    print("\nError: The optimization problem could not be solved. Status: {pulp.LpStatus[model.status]}")




The minimum required transportation cost (Z) is: €460.00

Optimal Transportation Plan (Flows > 0):
Source          Destination     Units Sent
----------------------------------------
S1             D2             5         
S1             D4             15        
S2             D2             10        
S2             D3             15        
S3             D1             10        
S3             D4             5         


##### **c) Fixed costs**

We introduce a binary variable $y_{ij}$ to track whether a route is active. 
1. New decision variable: $y_{ij} \in \{0,1\}$: equals 1 if goods are transported from source $i$ to destination $j$, and 0 otherwise. 
2. Objective function: the goal is to minimize the sum of variable costs and fixed costs. 
$$\text{Minimize } Z = \left( \sum_{i=1}^{3} \sum_{j=1}^{4} c_{ij} x_{ij} \right) + \left( \sum_{i=1}^{3} \sum_{j=1}^{4} 100 \cdot y_{ij} \right)$$
3. The binary variable $(y_{ij})$ is linked to the quantity $(x_{ij})$, using $M = 60$ (total supply/demand):
$$x_{ij} \le 60 \cdot y_{ij} \ \text{for all} \ i, j$$

In [4]:
# Data Definition
Sources = ['S1', 'S2', 'S3']
Destinations = ['D1', 'D2', 'D3', 'D4']

# Cost matrix
costs = {
    'S1': {'D1': 10, 'D2': 0, 'D3': 20, 'D4': 11}, 
    'S2': {'D1': 12, 'D2': 7, 'D3': 9, 'D4': 20}, 
    'S3': {'D1': 0, 'D2': 14, 'D3': 16, 'D4': 18}
}

M = 60
FIXED_COST = 100

# New model
model = pulp.LpProblem("ILP_with_Fixed_Costs", pulp.LpMinimize)

# Defining the binary variables
y = pulp.LpVariable.dicts("y",
                          ((i, j) for i in Sources for j in Destinations), 
                          cat='Binary') 
x = pulp.LpVariable.dicts("x",
                          ((i, j) for i in Sources for j in Destinations),
                          lowBound=0, cat='Continuous')

# Supply and demand constraints
for i in Sources:
    model += pulp.lpSum(x[(i,j)] for j in Destinations) <= supply[i], f"Supply_{i}"

for j in Destinations:
    model += pulp.lpSum(x[(i,j)] for i in Sources) >= demand[j], f"Demand_{j}"

# Redefining the objective
model += (pulp.lpSum(costs[i][j] * x[(i, j)] for i in Sources for j in Destinations)
          + pulp.lpSum(FIXED_COST * y[(i, j)] for i in Sources for j in Destinations)), "ILP_Total_Cost_with_Fixed"

# Adding in the constraints
for i in Sources:
    for j in Destinations: 
        model += x[(i, j)] <= M * y[(i, j)], f"M_Constraint_{i}_{j}"

# Solving the model
model.solve()

# Displaying the results
print("\n=== ILP SOLUTION (Fixed Costs) ===")
print(f"Solver Status: {pulp.LpStatus[model.status]}")

if pulp.LpStatus[model.status] == "Optimal":
    ilp_min_cost = pulp.value(model.objective)
    ilp_fixed_cost = pulp.value(pulp.lpSum(FIXED_COST * y[(i, j)] for i in Sources for j in Destinations))

    print("\n==================================================")
    print(f"Minimum total cost including fixed charges: €{ilp_min_cost:,.2f}")
    print(f"Fixed cost component: €{ilp_fixed_cost:,.2f}")
    print("==================================================")
    
    print("\nOptimal ILP Transport Plan (Flows > 0):")
    print("{:<15} {:<15} {:<10} {:<10}".format("Source", "Destination", "Units Sent", "Fixed Used")) 
    print("-" * 55)

    ilp_routes_used = 0
    for i in Sources:
        for j in Destinations:
            amount = pulp.value(x[(i, j)])
            is_used = pulp.value(y[(i, j)])
            
            if is_used > 0.5:
                ilp_routes_used += 1
                print("{:<15} {:<15} {:<10.0f} {:<10.0f}".format(i, j, amount, is_used))

    print(f"\nTotal number of distinct routes used: {ilp_routes_used}")



Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /opt/anaconda3/lib/python3.13/site-packages/pulp/apis/../solverdir/cbc/osx/i64/cbc /var/folders/25/0hf3c7d931q0rktyfwqp3g_w0000gn/T/8ac7cab73dfc429d91fe7e8f56eed251-pulp.mps -timeMode elapsed -branch -printingOptions all -solution /var/folders/25/0hf3c7d931q0rktyfwqp3g_w0000gn/T/8ac7cab73dfc429d91fe7e8f56eed251-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 24 COLUMNS
At line 119 RHS
At line 139 BOUNDS
At line 152 ENDATA
Problem MODEL has 19 rows, 24 columns and 48 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is 560 - 0.00 seconds
Cgl0004I processed model has 19 rows, 24 columns (12 integer (12 of which binary)) and 48 elements
Cbc0038I Initial state - 6 integers unsatisfied sum - 1.96667
Cbc0038I Pass   1: suminf.    0.73333 (2) obj. 868.333 iterations 4
Cbc0038I Solution found of 995
Cbc

The introduction of fixed costs compels the solver to adopt a different optimization strategy. The ILP solution (€995.00) is significantly higher due to the €500.00 in fixed costs (5 routes $\times$ €100). The solver prioritizes consolidating the total shipment onto a smaller number of distinct routes (5), even if it means using routes that have slightly higher variable costs (e.g., $c_{ij}$) than the minimum possible 6 routes used in the pure LP model. The goal shifts from finding the absolute cheapest per-unit cost to finding the cheapest set of routes to open.

#### **3. Inventory managment**
##### **a) *insert title***

##### **b) *insert title***

##### **c) *insert title***

#### **4. Single-Machine Scheduling**
##### **a) Integer linear optimization problem**

The goal is to find a production schedule that minimizes the total cost incurred from jobs exceeding their deadlines (total tardiness). 

**Decision variables**
- $t_j \ge 0$: the start time of job $j$ (continous). 
- $L_j \ge 0$: the tardiness (lateness cost) of job $j$. This is the time (in hours) job $j$ is completed after its due date (continous).  
- $x_{jk} \in \{0, 1\}$: a binary variable that determines the sequence:
    - $x_{jk} = 1$ if job $j$ is processed before job $k$.
    - $x_{jk} = 0$ otherwise. 

**Parameters**
- $D_j$: duration of job $j$
- $R_j$: release time of job $j$
- $E_j$: due date of job $j$
- $M$: a large constant (Big M), greater than the total processing time for all jobs ($M > 40$). 

**Objective function**

The total cost is minimized, where the cost rate is €1 per hour of tardiness. 
$$\text{Minimize } Z = \sum_{j=1}^{10} 1 \cdot L_j$$

**Constraints**

1. Sequencing 

With these constraints we ensure that only one job is running at a time (no overlap) and define the sequence using the Big M method. 
- Non-overlap Rule: for any pair of distinct jobs $j$ and $k$, if job $j$ is scheduled before job $k$ ($x_{jk} = 1$), then $k$ must start after $j$ is completed ($t_j + D_j \le t_k$). 
$$t_j + D_j \le t_k + M(1 - x_{jk}) \ \text{for all} \ j \ne k$$
- Mutual Exclusion: for any pair of distinct jobs, one must be processed before the other. 
$$x_{jk} + x_{kj} = 1 \ \text{for all} \ j < k$$

2. Timing constraints
- Release time: a job can't start before it's release time.
$$t_j \ge R_j \ \text{for all} \ j$$
- Tardiness definition: his constraint links the start time, duration and due date to the tardiness variable $L_j$. The minimization objective ensures $L_j$ is only positive if the completion time ($t_j + D_j$) exceeds the due date ($E_j$). 
$$L_j \ge t_j + D_j - E_j \ \text{for all} \ j$$

3. Variable constraints
- Non-negativity: $$t_j \ge 0, L_j \ge 0 \ \text{for all} \ j$$
- Binary: $$x_{jk} \in \{0, 1\} \ \text{for all} \ j \ne k$$

##### **b) Integer linear optimization model**
We define the parameters $D_j$ (Duration), $R_j$ (Release time) and $E_j$ (Due date) for all 10 jobs. We also need to determine the Big M value; the sum of $D_j$ is $4 + 5 + 3 + 5 + 7 + 0 + 3 + 2 + 10 = 40$. We choose $M = 100$ as a safe margin. 

In [5]:
# Job data
JOBS = list(range(1, 11)) # Jobs 1 to 10
M = 100 # Large constant

# Job parameters
D = {1: 4, 2: 5, 3: 3, 4: 5, 5: 7, 6: 1, 7: 0, 8: 3, 9: 2, 10: 10} # Duration
R = {1: 3, 2: 4, 3: 7, 4: 11, 5: 10, 6: 0, 7: 0, 8: 10, 9: 0, 10: 15} # Release time
E = {1: 11, 2: 12, 3: 20, 4: 25, 5: 20, 6: 10, 7: 30, 8: 30, 9: 10, 10: 20} # Due date
COST_RATE = 1 # Costs per hour of delay

# Model initialization
model = pulp.LpProblem("Single_Machine_Scheduling", pulp.LpMinimize)

# 1. Defining the decision variables
T = pulp.LpVariable.dicts("t", JOBS, lowBound=0, cat='Continuous') # Start time t_j
L = pulp.LpVariable.dicts("L", JOBS, lowBound=0, cat='Continuous') # Duration of delay L_j

# Binary ordering variable x_jk (only for j < k to avoid redundancy)
X = pulp.LpVariable.dicts("x", 
                          [(j, k) for j in JOBS for k in JOBS if j < k], 
                          cat='Binary')

We then implement the MILP-formulation. 

In [6]:
# 2. Objective function: minimize total delay costs
model += pulp.lpSum(COST_RATE * L[j] for j in JOBS), "Total_Tardiness_Cost"

# 3. Adding constraints
for j in JOBS:
    model += T[j] >= R[j], f"Release_Time_{j}"
    model += L[j] >= T[j] + D[j] - E[j], f"Tardiness_Definition_{j}" 

# 4. Non-overlap and Mutual Exclusion
for j in JOBS:
    for k in JOBS:
        if j < k:
            model += T[j] + D[j] <= T[k] + M * (1 - X[(j, k)]), f"Overlap_{j}_before_{k}"
            model += T[k] + D[k] <= T[j] + M * X[(j, k)], f"Overlap_{k}_before_{j}"

# Solving the model
model.solve()

# Reporting
print(f"Solver Status: {pulp.LpStatus[model.status]}")

if model.status == pulp.LpStatus[model.status] == "Optimal":
    total_tardiness = pulp.value(model.objective)
    print("\n==================================================")
    print(f"Minimum Total Delay costs: €{total_tardiness:.2f}")
    print("==================================================")

# Collect the results in a list for sorting
results = []
for j in JOBS:
    results.append({
        'Job': j,
        'Start_Time': pulp.value(T[j]), 
        'Duration': D[j], 
        'Completion_Time': pulp.value(T[j]) + D[j],
        'Due_Date': E[j],
        'Tardiness': pulp.value(L[j])
    })

# Sorting the starting time to decide on the optimal schedule
optimal_schedule = sorted(results, key=lambda x: x['Start_Time'])

print("\nOptimal Schedule:")
print ("{:<5} {:<15} {:<10} {:<15} {:<10} {:<15}".format("Job", "Start Time", "Duration", "Completion", "Due Date", "Tardiness"))
print("-" * 75)

# Printing the schedule and the corresponding values
for item in optimal_schedule:
    print("{:<5} {:<15.2f} {:<10.0f} {:<15.2f} {:<10.0f} {:<15.2f}".format(
        item['Job'], item['Start_Time'], item['Duration'], item['Completion_Time'], item['Due_Date'], item['Tardiness']))

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /opt/anaconda3/lib/python3.13/site-packages/pulp/apis/../solverdir/cbc/osx/i64/cbc /var/folders/25/0hf3c7d931q0rktyfwqp3g_w0000gn/T/735cec73ae2d47ec91bcb45b17cf3416-pulp.mps -timeMode elapsed -branch -printingOptions all -solution /var/folders/25/0hf3c7d931q0rktyfwqp3g_w0000gn/T/735cec73ae2d47ec91bcb45b17cf3416-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 115 COLUMNS
At line 516 RHS
At line 627 BOUNDS
At line 673 ENDATA
Problem MODEL has 110 rows, 65 columns and 300 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is 5 - 0.00 seconds
Cgl0004I processed model has 100 rows, 65 columns (45 integer (45 of which binary)) and 290 elements
Cbc0038I Initial state - 28 integers unsatisfied sum - 3.36
Cbc0038I Pass   1: suminf.    0.00000 (0) obj. 188 iterations 44
Cbc0038I Solution found of 188
Cbc00

The optimal schedule has a total tardiness of 24. Most jobs are scheduled without delay, but a few jobs incur delays due to machine capacity constraints and overlapping jobs. The MILP correctly minimized total delay whilst respecting release times and processing times. 

##### **c) Sieve changing time** 

This new introduced setup time of 1 hour must be added to the start time of the succeeding job if the sieve type changes. We first define the sieve types in the data structure. 

In [7]:
# 1. New data definition: sieve types

# Sieve 1: Jobs 1, 3, 4, 6, 10
# Sieve 2: Jobs 2, 5, 7, 8, 9
SIEVES = {1: 1, 2: 2, 3: 1, 4: 1, 5: 2, 6: 1, 7: 2, 8: 2, 9: 2, 10: 1}
SETUP_TIME = 1
M = 100 # Big M remains the same

# Function to determine if a setup is required between job j and k

def get_setup(j, k):
    """Returns 1 if the sieve type differs, otherwise 0."""
    return SETUP_TIME if SIEVES[j] != SIEVES[k] else 0

The crucial change is updating the Non-Overlap constraints by including the `SETUP_TIME` if $Sieve(j) \neq Sieve(k)$. The constraint $t_j + D_j \le t_k + M(1 - x_{jk})$ becomes:
$$t_j + D_j + Setup_{jk} \le t_k + M(1 - x_{jk})$$

In [8]:
# 2. Model re-initialization
model_c = pulp.LpProblem("Single_Machine_Scheduling_Setup", pulp.LpMinimize)
T_c = pulp.LpVariable.dicts("t", JOBS, lowBound=0, cat='Continuous')
L_c = pulp.LpVariable.dicts("L", JOBS, lowBound=0, cat='Continuous')
X_c = pulp.LpVariable.dicts("x", [(j, k) for j in JOBS for k in JOBS if j < k], cat='Binary')

# Objective function with the new variables
model_c += pulp.lpSum(COST_RATE * L_c[j] for j in JOBS), "Total_Tardiness_Cost"

# Release time constraints and tardiness definition
for j in JOBS:
    model_c += T_c[j] >= R[j], f"Release_Time_{j}"
    model_c += L_c[j] >= T_c[j] + D[j] - E[j], f"Tardiness_Def_{j}"
    model_c += L_c[j] >= 0, f"Tardiness_Nonneg_{j}"

# 3. Applying all constraints

# Release time and tardiness definition 
for j in JOBS: 
    for k in JOBS:
        if j != k:
            # Determine setup time for j -> k and k -> j
            setup_jk = get_setup(j, k)
            setup_kj = get_setup(k, j)
            x_var = X_c[(min(j, k), max(j, k))]
            
            # The switch variable (1-x_jk) is X_c if k precedes j, or (1-X_c) if j precedes k
            if j < k:
                model_c += T_c[j] + D[j] + setup_jk <= T_c[k] + M * (1 - x_var), f"Seq_{j}_before_{k}"
                model_c += T_c[k] + D[k] + setup_jk <= T_c[j] + M * (1 - x_var), f"Seq_{k}_before_{j}"
    
# Solving the model
model_c.solve()

# Reporting
print("\n=== SOLUTION Q4(c): Single-Machine Scheduling with Setup Time ===")
print(f"Solver Status: {pulp.LpStatus[model_c.status]}")

if pulp.LpStatus[model_c.status] == "Optimal":
    total_tardiness_c = pulp.value(model_c.objective)
    print("\n==================================================")
    print(f"Minimum Total Tardiness Cost (Q4c): €{total_tardiness_c:.2f}")
    print("==================================================")
    
    # Collecting and sorting the results by start time
    results_c = []
    for j in JOBS:
        results_c.append({
            'Job': j,
            'Sieve': SIEVES[j],
            'Start_Time': pulp.value(T_c[j]), 
            'Duration': D[j],
            'Tardiness': pulp.value(L_c[j])
    })
    
    optimal_schedule_c = sorted(results_c, key=lambda x: x['Start_Time'])
    
    print("\nOptimal Schedule (with Sieve Changeover):")
    print("{:<5} {:<8} {:<15} {:<15} {:<10}".format(
        "Job", "Sieve", "Start Time", "Completion", "Tardiness"))
    print("-" * 60)
    
    current_completion = 0

    for item in optimal_schedule_c:
        completion = item['Start_Time'] + item['Duration']
        print("{:<5} {:<8} {:<15.2f} {:<15.2f} {:<10.2f}".format(
            item['Job'], item['Sieve'], item['Start_Time'], completion, item['Tardiness']))


Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /opt/anaconda3/lib/python3.13/site-packages/pulp/apis/../solverdir/cbc/osx/i64/cbc /var/folders/25/0hf3c7d931q0rktyfwqp3g_w0000gn/T/4763374e24f54f20a987b36b96b1dd50-pulp.mps -timeMode elapsed -branch -printingOptions all -solution /var/folders/25/0hf3c7d931q0rktyfwqp3g_w0000gn/T/4763374e24f54f20a987b36b96b1dd50-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 125 COLUMNS
At line 536 RHS
At line 657 BOUNDS
At line 703 ENDATA
Problem MODEL has 120 rows, 65 columns and 310 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is 5 - 0.00 seconds
Cgl0004I processed model has 100 rows, 20 columns (0 integer (0 of which binary)) and 200 elements
Cbc3007W No integer variables - nothing to do
Cuts at root node changed objective from 5 to -1.79769e+308
Probing was tried 0 times and created 0 cuts of which 0 w

Compared to the schedule without setup times (part b), the new schedule includes sequence-dependent setup times, which change the optimal order of jobs. In part (b), the machine could switch freely between jobs because no extra time was required to change sieves. As a result, the solver simply ordered the jobs to minimize tardiness. In this part however, the model must also account for the extra time required when switching from one sieve type to another. This makes certain job sequences more expensive than others. The solver therefore prefers to group jobs with the same sieve type together, reduce the number of sieve changeovers and avoid switching back and forth between sieves. Because of this, some jobs start later than in the previous schedule and one job (job 10) becomes late, resulting in a total tardiness of 5. This lateness did not occur in part (b), where no setup times were counted. In conclusion, the introduction of setup times changes the optimal sequence: the order becomes less about due dates alone and more about finding a balance between meeting deadlines and minimizing sieve changeovers. 

##### **d) Quadratic Tardiness Costs**

The objective is changed from $\sum L_j$ to $\sum(L_j)^2$. We therefore must linearize this objective by replacing $L_j^2$ with a new variable $Y_j$, defined as a piecewise linear approximation of the function $f(L_j) = L_j^2$.

**Decision variables (per job $j$)**
- $Y_j \ge 0$: represents the quadratic cost $(L_j)^2$ (continuous). 
- $\lambda_{jk} \in [0, 1]$: weight for breakpoint $k$ (continuous). 

**Objective function**
$$\text{Minimize } Z = \sum_{j=1}^{10} Y_j$$

**New constraints**

For each job $j$, the following constraints link the original tardiness variable $L_j$ to the new linear components $Y_j$ and $\lambda_{jk}$. Let $Z_k$ be the tardiness breakpoint values and $f_k$ be the corresponding function values $Z_k^2$. 

1. Convex combination of tardiness ($L_j$):
$$L_j = \sum_k Z_k \cdot \lambda_jk$$

2. Linearization of cost ($Y_j$):
$$Y_j = \sum f_k \cdot \lambda_{jk}$$

3. Sum of weights:
$$\sum_k \lambda_{jk} = 1$$

Our new model (`model_d`) must include all the sequencing and timing constraints from 4(a) and 4(b), but also substitutes the objective and adds the linearization constraints. 